In [2]:
import os
import time
import requests
import pandas as pd
from unidecode import unidecode
from bs4 import BeautifulSoup
#from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout

# HISTORICO DE PARTIDAS

In [3]:
def historico_partidas(tabela_url, paginas):
  todas_partidas = []
  data = requests.get(tabela_url) # Pega o html (em forma de string) da pagina 'Tabela do Brasileirao'
  soup = BeautifulSoup(data.text) # Converte o String em html
  tabela_brasileirao = soup.select('table.stats_table')[0] # Seleciona dentro do HTML a tabela em si
  links_times = tabela_brasileirao.find_all('a') # Procura a Anchor que contem todos os links do time
  links_times = [link.get("href") for link in links_times] # Pega só os links em si
  links_times = [link for link in links_times if paginas[0] in link] # Escolhe o link que leva para a pagina de Stats do time
  times_urls = [f"https://fbref.com{link}" for link in links_times] # Transforma a string em URL

  # Para cada time faça:
  ## 1- Pega o nome do time
  ## 2- Pega as informações das abas (em ordem):
  ### Scores & Fixtures, Shooting, Goalkeeping, Passing, Pass Types, Goal and Shot Creation, Defensive Actions, Possession, Miscellaneous Stats
  ## 3- Transforma a tabela que esta em HTML em Dataframe
  for time_url in times_urls:
    nome_time = time_url.split('/')[-1].replace("-Stats", "")
    data = requests.get(time_url) # Pega o html (em forma de string) da pagina do time
    soup = BeautifulSoup(data.text)

    ## Scores & Fixtures
    partidas = pd.read_html(data.text, match= 'Scores & Fixtures')[0] # Transforma em dataframe o historico de partidas
    
    ## Shooting
    links = [l.get("href") for l in soup.find_all('a')]
    links = [l for l in links if l and paginas[1] in l]
    data_shooting = requests.get(f"https://fbref.com{links[0]}")
    shooting = pd.read_html(data_shooting.text, match= "Shooting")[0]
    shooting.columns = shooting.columns.droplevel()
    
    ## Goalkeeping
    links = [l.get("href") for l in soup.find_all('a')]
    links = [l for l in links if l and paginas[2] in l]
    data_goalkeeping = requests.get(f"https://fbref.com{links[0]}")
    goalkeeping = pd.read_html(data_goalkeeping.text, match= "Goalkeeping")[0]
    goalkeeping.columns = goalkeeping.columns.droplevel()
    
    ## Passing
    links = [l.get("href") for l in soup.find_all('a')]
    links = [l for l in links if l and paginas[3] in l]
    data_passing = requests.get(f"https://fbref.com{links[0]}")
    passing = pd.read_html(data_passing.text, match= "Passing")[0]
    passing.columns = passing.columns.droplevel()
    
    ## Pass Types
    links = [l.get("href") for l in soup.find_all('a')]
    links = [l for l in links if l and paginas[4] in l]
    data_passtype = requests.get(f"https://fbref.com{links[0]}")
    pass_types = pd.read_html(data_passtype.text, match= "Pass Types")[0]
    pass_types.columns = pass_types.columns.droplevel()
    
    ## Goal and Shot Creation
    links = [l.get("href") for l in soup.find_all('a')]
    links = [l for l in links if l and paginas[5] in l]
    data_shotcreation = requests.get(f"https://fbref.com{links[0]}")
    goal_shotcreation = pd.read_html(data_shotcreation.text, match= "Goal and Shot Creation")[0]
    goal_shotcreation.columns = goal_shotcreation.columns.droplevel()
    
    ## Defensive Actions
    links = [l.get("href") for l in soup.find_all('a')]
    links = [l for l in links if l and paginas[6] in l]
    data_defensive = requests.get(f"https://fbref.com{links[0]}")
    defensive = pd.read_html(data_defensive.text, match= "Defensive Actions")[0]
    defensive.columns = defensive.columns.droplevel()
    
    ## Possession
    ## Miscellaneous Stats

    # Merge das abas
    try:
      time_df = partidas.merge(
        shooting[['Date', 'Sh', 'SoT', 'SoT%', 'G/Sh', 'G/SoT', 'FK', 'PK', 'PKatt', 'npxG', 'npxG/Sh', 'G-xG', 'np:G-xG']], on= 'Date')
      time_df = time_df.merge(
        goalkeeping[['Date', 'SoTA', 'Saves', 'Save%', 'CS', 'PSxG', 'PSxG+/-']], on= 'Date')
      time_df = time_df.merge(
        passing[['Date', 'TotDist', 'PrgDist', 'Ast',	'xAG', 'xA', 'KP', '1/3', 'PPA', 'CrsPA', 'PrgP']], on= 'Date') 
      time_df = time_df.merge(
        pass_types[['Date', 'TB', 'Sw', 'Crs', 'Blocks']], on= 'Date')
      time_df = time_df.merge(
        goal_shotcreation[['Date', 'SCA', 'GCA']], on= 'Date')
      time_df = time_df.merge(
        defensive[['Date', 'TklW','Def 3rd','Mid 3rd','Att 3rd','Int','Err']], on= 'Date')
      #time_df = time_df.merge(
        #possession[['Date', ]], on= 'Date')
      #time_df = time_df.merge(
        #miscellaneous[['Date',]], on= 'Date')
    except ValueError:
      continue
    
    time_df = time_df[time_df["Comp"] == "Série A"] # Filtra o brasileirão
    time_df['Equipe'] = nome_time
    todas_partidas.append(time_df)
    time.sleep(10)

  historico = pd.concat(todas_partidas)
  return historico

tabela = "https://fbref.com/en/comps/24/Serie-A-Stats"
abas = ['/squads/', 'all_comps/shooting/', 'all_comps/keeper', 'all_comps/passing', 'all_comps/passing_types', 'all_comps/gca', 'all_comps/defense', 'all_comps/possession', 'all_comps/misc']
historico = historico_partidas(tabela, abas)

### Tratamento

In [8]:
def remover_acentos(texto):
    return unidecode(texto)

In [26]:
historico.drop(
  ['Notes','Captain', 'Formation', 'Referee', 'Match Report'], 
  axis= 'columns', inplace= True)
historico[['GF', 'GA']] = historico[['GF', 'GA']].astype(int)
historico['Opponent'] = historico['Opponent'].apply(remover_acentos)

substitutions = {
    'Botafogo-RJ': 'Botafogo (RJ)',
    'Gremio': 'Gremio',
    'Athletico-Paranaense': 'Ath Paranaense',
    'Atletico-Mineiro': 'Atletico Mineiro',
    'Sao-Paulo': 'Sao Paulo',
    'Vasco-da-Gama': 'Vasco da Gama',
    'America-MG': 'America (MG)'
}

for key, value in substitutions.items():
    historico['Equipe'] = historico['Equipe'].str.replace(unidecode(key), value, regex=True)

In [27]:
historico_casa = historico[historico['Venue'] == 'Home']

### Gerar planilha

In [28]:
data_ultimo_jogo = historico['Date'].max()
historico.to_excel(f'dados/historico/{data_ultimo_jogo}.xlsx', sheet_name='historico', index=False)
historico_casa.to_excel(f'dados/historico/casa-{data_ultimo_jogo}.xlsx', sheet_name='historico', index=False)

# TABELA

In [29]:
def tabela_brasileirao(tabela_url):
  data = requests.get(tabela_url)
  tabela = pd.read_html(data.text, match= 'Regular season')[0]
  return tabela

tabela_url = "https://fbref.com/en/comps/24/Serie-A-Stats"
tabela_df = tabela_brasileirao(tabela_url)

### Tratamento

In [30]:
tabela_df.drop(['Attendance', 'Top Team Scorer', 'Goalkeeper', 'Notes'], axis= 'columns', inplace= True)
tabela_df['xGConv'] = tabela_df['GF'] - tabela_df['xG']   #Conversão de xG
tabela_df['xGAConv'] = tabela_df['GA'] - tabela_df['xGA']   #Conversão de xGA
tabela_df['GF/MP'] = tabela_df['GF']/tabela_df['MP']    #Gols feitos por partida jogada
tabela_df['GA/MP'] = tabela_df['GA']/tabela_df['MP']    #Gols recebidos por partida jogada
tabela_df['Squad'] = tabela_df['Squad'].apply(remover_acentos)

### Gerar planilha

In [31]:
rodada = tabela_df['MP'].max()
tabela_df.to_excel(f'dados/tabela/rodada-{rodada}.xlsx', sheet_name='tabela', index=False)

### Historico por time

In [ ]:
rodada = tabela['MP'].max()
if not os.path.exists(f'dados/historico/times/rodada_{rodada}'):
  os.makedirs(f'dados/historico/times/rodada_{rodada}')

for time in historico_casa['Equipe']:
  df = historico_casa[historico_casa['Equipe'] == time]
  df = pd.concat([df, historico_casa[historico_casa['Opponent'] == time]])
  df.to_excel(f'dados/historico/times/rodada_{rodada}/{time}.xlsx')

# Proximas Rodadas

In [5]:
def proxs_rodadas(tabela_url):
  data = requests.get(tabela_url)
  tabela = pd.read_html(data.text, match= 'Scores & Fixtures')[0]
  return tabela

rodadas_url = "https://fbref.com/en/comps/24/schedule/Serie-A-Scores-and-Fixtures"
rodadas_df = proxs_rodadas(rodadas_url)

### Tratamento

In [10]:
rodadas_df = rodadas_df.loc[rodadas_df['Home'] != 'nan']
rodadas_df = rodadas_df.loc[rodadas_df['Away'] != 'nan']

In [12]:
rodadas_df['Home'] = rodadas_df['Home'].apply(remover_acentos)
rodadas_df['Away'] = rodadas_df['Away'].apply(remover_acentos)

substitutions = {
    'Botafogo-RJ': 'Botafogo (RJ)',
    'Gremio': 'Gremio',
    'Athletico-Paranaense': 'Ath Paranaense',
    'Atletico-Mineiro': 'Atletico Mineiro',
    'Sao-Paulo': 'Sao Paulo',
    'Vasco-da-Gama': 'Vasco da Gama',
    'America-MG': 'America (MG)'
}

for key, value in substitutions.items():
    rodadas_df['Home'] = rodadas_df['Home'].str.replace(unidecode(key), value, regex=True)
    rodadas_df['Away'] = rodadas_df['Away'].str.replace(unidecode(key), value, regex=True)

rodadas_df.drop(['Time', 'Attendance', 'Venue', 'Referee', 'Match Report', 'Notes'], axis= 1, inplace= True)
rodadas_df[['gols_casa', 'gols_fora']] = rodadas_df['Score'].str.split('–', expand=True)
rodadas_df.drop(['Score'], axis= 1, inplace= True)
rodadas_df.insert(5, 'gols_casa', rodadas_df.pop('gols_casa'))
rodadas_df.insert(6, 'gols_fora', rodadas_df.pop('gols_fora'))
rodadas_df = rodadas_df.rename(columns={'xG.1': 'xGFora', 'xG': 'xGCasa'})


KeyError: 'Score'

In [56]:
def convert_to_int(value):
    try:
        return int(value)
    except (ValueError, TypeError):
        return pd.NA
    
rodadas_df['gols_casa'] = rodadas_df['gols_casa'].apply(convert_to_int)
rodadas_df['gols_fora'] = rodadas_df['gols_fora'].apply(convert_to_int)

In [57]:
rodadas_df.to_excel(f'dados/previsoes/rodadas.xlsx', sheet_name='tabela', index=False)